In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import dask.dataframe as dd
import pandas as pd

# trade_df = dd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet')
trade_df = pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet')
# trade_df.compute()

In [ ]:
import pandas as pd
import numpy as np

def strat(df):
    price_df = df.set_index(['seconds_in_bucket'])['price'].fillna(
            method='pad')
    
    pred_rv = np.sqrt((price_df.apply(np.log).diff()**2).sum())
    return pred_rv

In [ ]:
# preds = trade_df.groupby(['time_id', 'stock_id']).apply(strat, meta=object).compute().rename('target')
preds = trade_df.groupby(['time_id', 'stock_id']).apply(strat).rename('target')
preds

In [ ]:
out_df = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/test.csv').merge(preds.reset_index(), how='left').drop(['time_id', 'stock_id'], axis=1)

out_df.fillna({'target': out_df.target.mean()}, inplace=True)

out_df.to_csv('submission.csv', index=False)